### Import library

In [1]:
import requests
import pandas as pd
import json
from pprint import PrettyPrinter
import tensorflow as tf
import base64

### Inisialisasi PrettyPrinter untuk output yang lebih rapi

In [2]:
pp = PrettyPrinter()

### URL TensorFlow Serving endpoint

In [3]:
TF_SERVING_URL = "http://localhost:8501/v1/models/stress-model:predict"

### Convert text to json

In [4]:
def text_to_json(text):
    byteList = {
        'text': tf.train.Feature(bytes_list=tf.train.BytesList(value=[bytes(text, 'utf-8')]))
    }

    serialToString = tf.train.Example(
        features = tf.train.Features(feature=byteList)
    ).SerializeToString()

    resultJson = [
        {
            'examples': {
                'b64': base64.b64encode(serialToString).decode()
            }
        }
    ]

    return json.dumps({
        'signature_name': 'serving_default',
        'instances': resultJson
    })


In [5]:
input = "October is Domestic Violence Awareness Month and I am a domestic violence survivor who is still struggling, even after over four years. Lately I have been feeling very angry. Angry that my abusive ex received no real consequences for his actions. This man abused me in all manners: physically, sexually, emotionally, verbally, financially, etc. I was granted a restraining order against him (and it was renewed a year later) but I was unable to press criminal charges against him because I didn’t have enough evidence to have a case."

In [6]:
payloadJson = text_to_json(input)

In [7]:
try:
    response = requests.post(TF_SERVING_URL, data=payloadJson)
    response.raise_for_status()  # Raise an error for HTTP issues
    predictions = response.json() 
    # Output hasil prediksi
    if predictions:
        print("\nHasil prediksi:")
        if predictions["predictions"][0][0]> 0.6:
            print("Stress 🙁")
        else:
            print("Tidak Stress 🙂")
    else:
        print("Error: No predictions found in response.")


    
except requests.exceptions.RequestException as e:
    print(f"Error: {e}")


Hasil prediksi:
Stress 🙁
